# Test model

### original file

https://github.com/openai/gpt-2-output-dataset/blob/master/baseline.py

## Before running this notebook

1. Create /output folder
   1. Insert all crawled dataset(csv)
   1. Rename them as same with GPT dataset files.
1. Create /log folder

## Difference between baseline

- Using Word2Vec instead of TF-IDF Vectorizer
- Using Gridsearch to find best parameter

In [1]:
!pip install gensim
!pip install scikit-learn

In [2]:
# import packages

import os
import csv
import json

import numpy as np

from scipy import sparse

from sklearn.model_selection import PredefinedSplit, GridSearchCV, train_test_split, KFold
from sklearn.linear_model import LogisticRegression

from gensim.models import Word2Vec

In [3]:
# create tokenizer
# example code from https://github.com/SKT-AI/KoGPT2

from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>",
)

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
SHORT_MODE = True

In [5]:
# load data + preprocessing

def load_data(data_dir, crawled_dir, source):
    path = os.path.join(data_dir, "{}.csv".format(source))
    crawled_path = os.path.join(crawled_dir, "{}.csv".format(source))
    dataset = list(csv.reader(open(path, encoding="utf8")))
    crawled_dataset = list(csv.reader(open(crawled_path, encoding="cp949")))
    n = len(dataset)
    
    length = min(50, n) if SHORT_MODE else n

    texts = []
    labels = [1, 0] * length

    for data in dataset[:length]:
        idx = int(round(float(data[0])))
        tokens = tokenizer.tokenize(data[5])
        texts.append(' '.join(tokens))
        tokens = tokenizer.tokenize(crawled_dataset[idx][4])
        texts.append(' '.join(tokens))
    return texts, labels

In [6]:
# main function

def main(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_train, texts_test, labels_train, labels_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    n_train, n_test = len(texts_train), len(texts_test)
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

    model = LogisticRegression(max_iter=20)
    params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
    split = PredefinedSplit([-1]*n_train+[0]*n_test)
    search = GridSearchCV(model, params, cv=split, refit=False)
    search.fit(sparse.vstack([train_features, test_features]), labels_train + labels_test)
    model = model.set_params(**search.best_params_)
    model.fit(train_features, labels_train)
    test_accuracy = model.score(test_features, labels_test) * 100.0

    result = model.predict(test_features)
    result_proba = model.predict_proba(test_features)
    result_log_proba = model.predict_log_proba(test_features)
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_test, result_log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
        "ce_loss": ce_loss,
        "param": search.best_params_,
        "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print(data)
    json.dump(data, open(os.path.join(log_dir, "result.json"), "w"), indent=4)

In [7]:
# main function

def main_overfit(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_train, texts_test, labels_train, labels_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

    model = LogisticRegression(max_iter=10000, penalty=None)
    model.fit(train_features, labels_train)
    test_accuracy = model.score(test_features, labels_test) * 100.0

    result = model.predict(test_features)
    result_proba = model.predict_proba(test_features)
    result_log_proba = model.predict_log_proba(test_features)
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_test, result_log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
        "ce_loss": ce_loss,
        "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print(data)
    json.dump(data, open(os.path.join(log_dir, "result_overfit.json"), "w"), indent=4)

In [8]:
# main function

def main_kfold(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic)
        texts_list.extend(texts)
        labels_list.extend(labels)
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    
    
    kf = KFold(n_splits=5)
    
    for idx, (train, test) in enumerate(kf.split(texts_list)):
        texts_train, texts_test = [texts_list[x] for x in train], [texts_list[x] for x in test]
        labels_train, labels_test = [labels_list[x] for x in train], [labels_list[x] for x in test]

        n_train, n_test = len(train), len(test)
        
        train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
        test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

        model = LogisticRegression(max_iter=100)
        params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
        split = PredefinedSplit([-1]*n_train+[0]*n_test)
        search = GridSearchCV(model, params, cv=split, refit=False)
        search.fit(sparse.vstack([train_features, test_features]), labels_train + labels_test)
        model = model.set_params(**search.best_params_)
        model.fit(train_features, labels_train)
        test_accuracy = model.score(test_features, labels_test) * 100.0

        result = model.predict(test_features)
        result_proba = model.predict_proba(test_features)
        result_log_proba = model.predict_log_proba(test_features)
        kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
        for res, pred in zip(result, labels_test):
            if res == 1:
                kind["tp" if res == pred else "fp"] += 1
            else:
                kind["tn" if res == pred else "fn"] += 1
        precision = kind["tp"] / (kind["tp"] + kind["fp"])
        recall = kind["tp"] / (kind["tp"] + kind["fn"])

        ce_loss = 0
        for label, value in zip(labels_test, result_log_proba):
            ce_loss -= label * value[1] + (1 - label) * value[0]
        ce_loss /= len(labels_test)
        data = {
            "test_accuracy": test_accuracy,
            "test_precision": precision,
            "test_recall": recall,
            "F_score": 2 * precision * recall / (precision + recall),
            "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
            "ce_loss": ce_loss,
            "param": search.best_params_,
            "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
        }
        print(data)
        json.dump(data, open(os.path.join(log_dir, "result{}.json".format(idx)), "w"), indent=4)

In [9]:
# run main function

main(train_test_ratio=0.2)
# main_overfit(train_test_ratio=0.2)
# main_kfold()

{'test_accuracy': 85.83333333333333, 'test_precision': 0.9122807017543859, 'test_recall': 0.8125, 'F_score': 0.859504132231405, 'mse_loss': 0.5380300978392314, 'ce_loss': 0.4264432430521929, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.5632702353288629), (0, 0, 0.19871362267397785), (0, 0, 0.12707697698564305), (0, 0, 0.01617545614247215), (0, 0, 0.24381743954360924), (1, 1, 0.8835580692029219), (1, 1, 0.5609684354253088), (0, 0, 0.2700504124219628), (1, 1, 0.8929476844287356), (0, 0, 0.03227692911773108), (0, 0, 0.1307951205593235), (1, 0, 0.22470776025905675), (0, 0, 0.34078161291801656), (1, 1, 0.9607576738825248), (0, 0, 0.07432146444400861), (1, 1, 0.6506884584456407), (1, 1, 0.6563874795070928), (1, 1, 0.8337588128354759), (1, 1, 0.8412751894419865), (1, 0, 0.23824859781465577), (1, 1, 0.6184054848628312), (0, 0, 0.1597761865832577), (0, 0, 0.18779994738087652), (1, 1, 0.9528871641866076), (1, 0, 0.44264756058889776), (0, 0, 0.251052431033892), (1, 1, 0.9218332001720144), (0

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti